In [1]:
# from flask import Flask, render_template, redirect
from flask import Flask, jsonify, request
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from flask_sqlalchemy import SQLAlchemy
import pandas as pd
import os

In [2]:
file_path = os.path.abspath(os.getcwd()) + "/bb_app/db/belly_button_biodiversity.sqlite"
# engine = create_engine('sqlite:///'+ file_path)
# # engine = create_engine("sqlite:///db/belly_button_biodiversity.sqlite")
# inspector = inspect(engine)

**@app.route('/names')**

In [ ]:
def find_col_names(eng, table_name):
    inspector = inspect(eng)
    tables = {}
    
    for table in inspector.get_table_names():
        columns = inspector.get_columns(table)
        names = []
        for c in columns:
            names.append(c['name'])
            tables[table] = names
    
    column_names = tables[table_name]        
    del column_names[:1] #del first name(index)
    
    return column_names

In [ ]:
tables = {}
    
for table in inspector.get_table_names():
    columns = inspector.get_columns(table)
    names = []
    for c in columns:
        names.append(c['name'])
        tables[table] = names

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [ ]:
Samples_metadata = Base.classes.samples_metadata
Samples = Base.classes.samples
Otu = Base.classes.otu

**@app.route('/otu')**

In [ ]:
bact_tuples = session.query(Otu.lowest_taxonomic_unit_found).all()
bacteries = []

In [ ]:
for b_tuple in bact_tuples:
    for bact in b_tuple:
        bacteries.append(bact)
#bacteries = [bact for b_tuple in bact_tuples for bact in b_tuple]

**@app.route('/metadata/<sample>')**

In [ ]:
def metasample(eng, sample):
    if sample in find_col_names(eng, 'samples'):
        form_sample = int(sample[3:])
    
        keys = ['AGE', 'BBTYPE', 'ETHNICITY', 'GENDER', 'LOCATION', 'SAMPLEID']
        values = [f for s in eng.execute(f'SELECT * FROM samples_metadata where SAMPLEID = {form_sample}').\
          fetchall() for f in s]
    
        sample_dict = {}
        cols = inspector.get_columns('samples_metadata')
        for x in range(len(cols)):
            if cols[x]['name'] in keys:
                sample_dict[cols[x]['name']] = values[x]
        return sample_dict
    else:
        return f'There is no sample named {sample} in our dataset'

In [ ]:
keys = ['AGE', 'BBTYPE', 'ETHNICITY', 'GENDER', 'LOCATION', 'SAMPLEID']
q = 'BB_1265'
r = session.query(Samples_metadata.AGE, Samples_metadata.BBTYPE, Samples_metadata.ETHNICITY,\
                 Samples_metadata.GENDER, Samples_metadata.LOCATION, Samples_metadata.SAMPLEID).\
                filter(Samples_metadata.SAMPLEID == q).all()
values = [y for x in r for y in x]
sample_dict = {}
for x in range(len(keys)):
    sample_dict[keys[x]] = values[x]

In [ ]:
values = [f for s in engine.execute(f'SELECT * FROM samples_metadata where SAMPLEID = {q}').\
          fetchall() for f in s]

In [ ]:
sample_dict2 = {}
cols = inspector.get_columns('samples_metadata')
for x in range(len(cols)):
    if cols[x]['name'] in keys:
        sample_dict2[cols[x]['name']] = values[x]

In [ ]:
int(session.query(Samples_metadata.WFREQ).filter(Samples_metadata.SAMPLEID == 940).all()[0][0])
fetch = engine.execute(f'select "{t}", otu_id from samples where "{t}" > 0 order by "{t}" desc' ).fetchall()
val, ids = zip(*fetch)
values_otu = [dict(otu_ids=list(ids)), dict(sample_values=list(val))]
pd.DataFrame([f for f in fetch], columns=['sample_values', 'otu_ids'])